In [73]:
#!pip3 install summa
#!pip3 install python-rake
#!pip3 install scikit-learn scipy matplotlib numpy
import pandas as pd
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
from pymorphy2.tokenizers import simple_word_tokenize
from string import punctuation
punctuation = punctuation + '—–«»·№˚¼°'
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import summa
from summa import keywords
import RAKE
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1091)>


## Источник
Данные были взяты взяты из научного журнала «Постнаука» из следующих статей:

https://postnauka.ru/faq/155130

https://postnauka.ru/longreads/156624

https://postnauka.ru/talks/156733

https://postnauka.ru/longreads/156663

https://postnauka.ru/longreads/156517

Ключевые слова представлены в виде тегов к статьям

In [74]:
df = pd.read_csv('data.csv', error_bad_lines=False)

/Users/elizavetashu/Library/Python/3.7/lib/python/site-packages/IPython/core/interactiveshell.py:3417: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [75]:
df

,text,keywords,my_keywords
0,Психолог Марк Ламли о нарушении деятельности ч...,"биология, психология, болезнь, боль, болезни, ...","боль, генетика, хроническая боль, тревожность,..."
1,В привычном понимании термин «психосоматика» с...,"психология, кардиология, психотерапия, медицин...","психосоматика, болезнь, расстройства, гипертон..."
2,"Обучение происходит продуктивнее, если подходы...","психология, мотивация, обучение, обучение с уч...","обучение, мотивация, навыки, умения, мышление,..."
3,Мирный атом — это одна из основных надежд чело...,"атом, топливо, ядерная физика, топливные элеме...","атом, ядерная энергетика, ядерное топливо, ура..."
4,В последние годы интерес к правильному питанию...,"медицина, белки, питание, еда, калории, углево...","питание, медицина, диета, белки, жиры, углевод..."


In [76]:
def preprocessing(text):
    text = text.lower()
    for i in text:
        if i in punctuation:
            text = text.replace(i, '')
    text = text.replace('\n', ' ')
    text = text.replace('       ', ' ')
    text = text.replace('   ', ' ')
    text = text.replace('  ', ' ')
    text = text.replace('\ufeff', '')
    return text

In [77]:
def normalize_text(text):
    lemmas = []
    for t in simple_word_tokenize(text):
        lemmas.append(
            morph.parse(t)[0].normal_form
        )
    return ' '.join(lemmas)

In [78]:
df['text'] = df['text'].apply(preprocessing)

In [79]:
tokens_num = []
for i in df['text']:
    tokens_num.append(len(i.split(' ')))
df['tokens_num'] = tokens_num

In [80]:
df

,text,keywords,my_keywords,tokens_num
0,психолог марк ламли о нарушении деятельности ч...,"биология, психология, болезнь, боль, болезни, ...","боль, генетика, хроническая боль, тревожность,...",2689
1,в привычном понимании термин психосоматика ста...,"психология, кардиология, психотерапия, медицин...","психосоматика, болезнь, расстройства, гипертон...",460
2,обучение происходит продуктивнее если подходы ...,"психология, мотивация, обучение, обучение с уч...","обучение, мотивация, навыки, умения, мышление,...",1836
3,мирный атом это одна из основных надежд челове...,"атом, топливо, ядерная физика, топливные элеме...","атом, ядерная энергетика, ядерное топливо, ура...",2840
4,в последние годы интерес к правильному питанию...,"медицина, белки, питание, еда, калории, углево...","питание, медицина, диета, белки, жиры, углевод...",1977


## Эталонные ключевые слова

In [81]:
keywords_intersection = [list(set(df['keywords'][i].split(', ')) & set(df['my_keywords'][i].split(', '))) for i in range(len(df))]
keywords_intersection

[['боль'],
 ['психосоматика'],
 ['обучение', 'мотивация'],
 ['атом', 'уран'],
 ['углеводы', 'белки', 'калории', 'питание', 'медицина']]

Кажется, что пересечение ключевых слов довольно маленькое и стоит использовать их объединение

In [82]:
keywords_combining = [list(set(df['keywords'][i].split(', ')) | set(df['my_keywords'][i].split(', '))) for i in range(len(df))]
keywords_combining

[['журнал',
  'эмоции',
  'хроническая боль',
  'тревожность',
  'биология',
  'лечение',
  'болезни',
  'генетика',
  'боль',
  'болезнь',
  'психология',
  'естественные науки'],
 ['журнал',
  'диагноз',
  'биопсихосоциальный подход',
  'медицинская диагностика',
  'гипертония',
  'кардиология',
  'психосоматика',
  'психотерапия',
  'расстройства',
  'болезнь',
  'психология',
  'лекарства',
  'когнитивные науки',
  'терапия'],
 ['обучение',
  'журнал',
  'социальные науки',
  'обучение с учителем',
  'дети',
  'навыки',
  'когнитивные способности',
  'мышление',
  'взрослые',
  'мотивация',
  'психология',
  'умения'],
 ['журнал',
  'ядерная физика',
  'нейтроны',
  'толерантное топливо',
  'уран',
  'топливные элементы',
  'отходы',
  'ядерное топливо',
  'атом',
  'ядерная энергетика',
  'топливо'],
 ['журнал',
  'витамины',
  'жиры',
  'углеводы',
  'белки',
  'калории',
  'еда',
  'диета',
  'питание',
  'медицина',
  'естественные науки']]

In [83]:
df['best_keywords'] = keywords_combining

## Rake

In [84]:
stop = stopwords.words('russian')
stop.extend(('это', 'также', 'весь', 'который', 'изза', 'нам', 'которые', 'изз'))
rake = RAKE.Rake(stop)

In [85]:
def rake_keywords(text):
    text_keywords = rake.run(normalize_text(text), maxWords=3, minFrequency=2)
    return text_keywords

In [86]:
rake_keywords_list = []
for text in df['text']:
    rake_keywords_list.append(rake_keywords(text))

In [87]:
rake_keywords_list

[[('минимум шесть месяц', 8.666666666666666),
  ('нейронный путь задействовать', 8.0),
  ('хронический боль', 4.981167608286253),
  ('свой тело', 4.375),
  ('субъективный переживание', 4.333333333333334),
  ('нормальный жизнь', 4.333333333333334),
  ('генерирование эмоция', 3.8),
  ('человек', 2.3636363636363638),
  ('боль', 2.2033898305084745),
  ('травма', 2.1666666666666665),
  ('некоторый', 2.142857142857143),
  ('пациент', 2.1176470588235294),
  ('тело', 2.0),
  ('помогать', 2.0),
  ('подход', 2.0),
  ('проблема', 1.9166666666666667),
  ('мочь', 1.9),
  ('связать', 1.875),
  ('мозг', 1.8),
  ('случай', 1.8),
  ('работа', 1.75),
  ('расстройство', 1.75),
  ('фибромиалгия', 1.6666666666666667),
  ('работать', 1.6666666666666667),
  ('нужно', 1.6666666666666667),
  ('стресс', 1.6666666666666667),
  ('наблюдаться', 1.6666666666666667),
  ('влиять', 1.6),
  ('сигнал', 1.6),
  ('рассматривать', 1.5),
  ('заживать', 1.3333333333333333),
  ('поведение', 1.3333333333333333),
  ('страна', 1

In [88]:
df['rake_keywords'] = rake_keywords_list
df['rake_keywords'] = df['rake_keywords'].apply(lambda x: [i[0] for i in x])

## TextRank

In [89]:
def textrank_keywords(text):
    text_keywords = keywords.keywords(normalize_text(text), language='russian', additional_stopwords=stop, scores=True)
    return text_keywords

In [90]:
textrank_keywords_list = []
for text in df['text']:
    textrank_keywords_list.append(textrank_keywords(text))

In [91]:
textrank_keywords_list

[[('боль', 0.6052841213451229),
  ('пациент', 0.19030954097625394),
  ('мочь', 0.1419980708314486),
  ('деятельность человек', 0.1354769963513329),
  ('хронический', 0.11783101113204497),
  ('эмоциональный', 0.10910090513882778),
  ('частый', 0.10600636670779036),
  ('часто', 0.10600636670779036),
  ('часть', 0.10600636670779036),
  ('некоторый', 0.10008079449592233),
  ('свой', 0.09372524365557595),
  ('определённый проблема', 0.09050815324242825),
  ('хороший', 0.0900311716370651),
  ('лечение', 0.08935696289766988),
  ('травма', 0.08754957856231904),
  ('связать', 0.08562195135766851),
  ('связанный', 0.08562195135766851),
  ('заболевание', 0.08414596956523887),
  ('болевой ощущение', 0.08372627509112222),
  ('конкретный', 0.07592074154105495),
  ('конкретно', 0.07592074154105495),
  ('необходимо', 0.07382620167365841),
  ('необходимый', 0.07382620167365841),
  ('фактор', 0.07197543840988804),
  ('разный', 0.07092831511146329),
  ('расстройство', 0.07053838642108673),
  ('многие', 0

In [92]:
df['textrank_keywords'] = textrank_keywords_list
df['textrank_keywords'] = df['textrank_keywords'].apply(lambda x: [i[0] for i in x])

## TF-IDF

In [93]:
vectorizer = TfidfVectorizer(stop_words=stop)

In [94]:
def tfidf_keywords(text):
    text = normalize_text(text)
    X = vectorizer.fit_transform([text])
    feature_names = np.array(vectorizer.get_feature_names())
    denselist = np.array(X.todense())
    indices = denselist.argsort()[0]
    text_keywords = list(feature_names[indices[::-1][:15]])
    return text_keywords

In [95]:
tfidf_keywords_list = []
for text in df['text']:
    tfidf_keywords_list.append(tfidf_keywords(text))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [96]:
tfidf_keywords_list

[['боль',
  'хронический',
  'человек',
  'пациент',
  'болевой',
  'мочь',
  'связать',
  'некоторый',
  'проблема',
  'мозг',
  'ощущение',
  'подход',
  'лечение',
  'хороший',
  'свой'],
 ['заболевание',
  'приводить',
  'расстройство',
  'человек',
  'соматический',
  'гипертония',
  'фактор',
  'случай',
  'болезнь',
  'психосоматический',
  'проблема',
  'стресс',
  'психологический',
  'развитие',
  'год'],
 ['обучение',
  'ребёнок',
  'действие',
  'когнитивный',
  'способ',
  'мочь',
  'способность',
  'всё',
  'взрослый',
  'мышление',
  'хороший',
  'развитие',
  'собственный',
  'процесс',
  'этап'],
 ['топливо',
  'реактор',
  'ядерный',
  'уран',
  'плутоний',
  'нейтрон',
  'однако',
  'деление',
  'год',
  'энергетика',
  'энергия',
  'материал',
  'ядро',
  'делиться',
  'большой'],
 ['углевод',
  'жир',
  'организм',
  'человек',
  'белок',
  'энергия',
  'продукт',
  'количество',
  'калория',
  'грамм',
  'должный',
  'пища',
  'простой',
  'источник',
  'питание']

In [97]:
df['tfidf_keywords'] = tfidf_keywords_list

In [98]:
df

,text,keywords,my_keywords,tokens_num,best_keywords,rake_keywords,textrank_keywords,tfidf_keywords
0,психолог марк ламли о нарушении деятельности ч...,"биология, психология, болезнь, боль, болезни, ...","боль, генетика, хроническая боль, тревожность,...",2689,"[журнал, эмоции, хроническая боль, тревожность...","[минимум шесть месяц, нейронный путь задейство...","[боль, пациент, мочь, деятельность человек, хр...","[боль, хронический, человек, пациент, болевой,..."
1,в привычном понимании термин психосоматика ста...,"психология, кардиология, психотерапия, медицин...","психосоматика, болезнь, расстройства, гипертон...",460,"[журнал, диагноз, биопсихосоциальный подход, м...",[гипертония],"[фактор, человек, расстройство, соматический з...","[заболевание, приводить, расстройство, человек..."
2,обучение происходит продуктивнее если подходы ...,"психология, мотивация, обучение, обучение с уч...","обучение, мотивация, навыки, умения, мышление,...",1836,"[обучение, журнал, социальные науки, обучение ...","[способ обучение, ребёнок, способ, мочь, работ...","[ребёнок, взрослый, собственный, действие, эта...","[обучение, ребёнок, действие, когнитивный, спо..."
3,мирный атом это одна из основных надежд челове...,"атом, топливо, ядерная физика, топливные элеме...","атом, ядерная энергетика, ядерное топливо, ура...",2840,"[журнал, ядерная физика, нейтроны, толерантное...","[поддержание цепной реакция, быстрый нейтрон, ...","[ядерный топливо, уран, урана, плутоний, плуто...","[топливо, реактор, ядерный, уран, плутоний, не..."
4,в последние годы интерес к правильному питанию...,"медицина, белки, питание, еда, калории, углево...","питание, медицина, диета, белки, жиры, углевод...",1977,"[журнал, витамины, жиры, углеводы, белки, кало...","[быстрый углевод, первый очередь, углевод, жир...","[углевод, человек, количество, белок, организм...","[углевод, жир, организм, человек, белок, энерг..."


## Морфологические/синтаксические шаблоны для ключевых слов и фраз
Возьмем следующие шаблоны ключевых слов и фраз:

In [99]:
templates = [['NOUN'], ['ADJF'], ['ADJF', 'NOUN']]

In [100]:
def templates_keywords(keywords):
    temp_keywords = []
    for keyword in keywords:
        keyword_pos = []
        for word in keyword.split(' '):
            keyword_pos.append(morph.parse(word)[0].tag.POS)
        if keyword_pos in templates:
            temp_keywords.append(keyword)
    return temp_keywords

In [101]:
templates_rake_keywords = []
for j in df['rake_keywords']:
    templates_rake_keywords.append(templates_keywords(j))
df['templates_rake_keywords'] = templates_rake_keywords

In [102]:
templates_textrank_keywords = []
for j in df['textrank_keywords']:
    templates_textrank_keywords.append(templates_keywords(j))
df['templates_textrank_keywords'] = templates_textrank_keywords

In [103]:
templates_tfidf_keywords = []
for j in df['tfidf_keywords']:
    templates_tfidf_keywords.append(templates_keywords(j))
df['templates_tfidf_keywords'] = templates_tfidf_keywords

In [104]:
df

,text,keywords,my_keywords,tokens_num,best_keywords,rake_keywords,textrank_keywords,tfidf_keywords,templates_rake_keywords,templates_textrank_keywords,templates_tfidf_keywords
0,психолог марк ламли о нарушении деятельности ч...,"биология, психология, болезнь, боль, болезни, ...","боль, генетика, хроническая боль, тревожность,...",2689,"[журнал, эмоции, хроническая боль, тревожность...","[минимум шесть месяц, нейронный путь задейство...","[боль, пациент, мочь, деятельность человек, хр...","[боль, хронический, человек, пациент, болевой,...","[хронический боль, свой тело, субъективный пер...","[боль, пациент, мочь, хронический, эмоциональн...","[боль, хронический, человек, пациент, болевой,..."
1,в привычном понимании термин психосоматика ста...,"психология, кардиология, психотерапия, медицин...","психосоматика, болезнь, расстройства, гипертон...",460,"[журнал, диагноз, биопсихосоциальный подход, м...",[гипертония],"[фактор, человек, расстройство, соматический з...","[заболевание, приводить, расстройство, человек...",[гипертония],"[фактор, человек, расстройство, соматический з...","[заболевание, расстройство, человек, соматичес..."
2,обучение происходит продуктивнее если подходы ...,"психология, мотивация, обучение, обучение с уч...","обучение, мотивация, навыки, умения, мышление,...",1836,"[обучение, журнал, социальные науки, обучение ...","[способ обучение, ребёнок, способ, мочь, работ...","[ребёнок, взрослый, собственный, действие, эта...","[обучение, ребёнок, действие, когнитивный, спо...","[ребёнок, способ, мочь, подход, ситуация, зави...","[ребёнок, взрослый, собственный, действие, эта...","[обучение, ребёнок, действие, когнитивный, спо..."
3,мирный атом это одна из основных надежд челове...,"атом, топливо, ядерная физика, топливные элеме...","атом, ядерная энергетика, ядерное топливо, ура...",2840,"[журнал, ядерная физика, нейтроны, толерантное...","[поддержание цепной реакция, быстрый нейтрон, ...","[ядерный топливо, уран, урана, плутоний, плуто...","[топливо, реактор, ядерный, уран, плутоний, не...","[быстрый нейтрон, атомный энергетика, повышенн...","[ядерный топливо, уран, урана, плутоний, плуто...","[топливо, реактор, ядерный, уран, плутоний, не..."
4,в последние годы интерес к правильному питанию...,"медицина, белки, питание, еда, калории, углево...","питание, медицина, диета, белки, жиры, углевод...",1977,"[журнал, витамины, жиры, углеводы, белки, кало...","[быстрый углевод, первый очередь, углевод, жир...","[углевод, человек, количество, белок, организм...","[углевод, жир, организм, человек, белок, энерг...","[быстрый углевод, первый очередь, углевод, жир...","[углевод, человек, количество, белок, организм...","[углевод, жир, организм, человек, белок, энерг..."


 ## Оценка точности, полноты, F-меры 

In [105]:
df['best_keywords'] = df['best_keywords'].apply(lambda x: [preprocessing(i) for i in x])
df['best_keywords'] = df['best_keywords'].apply(lambda x: [normalize_text(i) for i in x])
df['best_keywords']

0    [журнал, эмоция, хронический боль, тревожность...
1    [журнал, диагноз, биопсихосоциальный подход, м...
2    [обучение, журнал, социальный наука, обучение ...
3    [журнал, ядерный физика, нейтрон, толерантный ...
4    [журнал, витамин, жир, углевод, белка, калория...
Name: best_keywords, dtype: object

Применение шаблона к эталонным ключевым словам

In [106]:
templates_best_keywords = []
for j in df['best_keywords']:
    templates_best_keywords.append(templates_keywords(j))
df['templates_best_keywords'] = templates_best_keywords

In [107]:
df

,text,keywords,my_keywords,tokens_num,best_keywords,rake_keywords,textrank_keywords,tfidf_keywords,templates_rake_keywords,templates_textrank_keywords,templates_tfidf_keywords,templates_best_keywords
0,психолог марк ламли о нарушении деятельности ч...,"биология, психология, болезнь, боль, болезни, ...","боль, генетика, хроническая боль, тревожность,...",2689,"[журнал, эмоция, хронический боль, тревожность...","[минимум шесть месяц, нейронный путь задейство...","[боль, пациент, мочь, деятельность человек, хр...","[боль, хронический, человек, пациент, болевой,...","[хронический боль, свой тело, субъективный пер...","[боль, пациент, мочь, хронический, эмоциональн...","[боль, хронический, человек, пациент, болевой,...","[журнал, эмоция, хронический боль, тревожность..."
1,в привычном понимании термин психосоматика ста...,"психология, кардиология, психотерапия, медицин...","психосоматика, болезнь, расстройства, гипертон...",460,"[журнал, диагноз, биопсихосоциальный подход, м...",[гипертония],"[фактор, человек, расстройство, соматический з...","[заболевание, приводить, расстройство, человек...",[гипертония],"[фактор, человек, расстройство, соматический з...","[заболевание, расстройство, человек, соматичес...","[журнал, диагноз, биопсихосоциальный подход, м..."
2,обучение происходит продуктивнее если подходы ...,"психология, мотивация, обучение, обучение с уч...","обучение, мотивация, навыки, умения, мышление,...",1836,"[обучение, журнал, социальный наука, обучение ...","[способ обучение, ребёнок, способ, мочь, работ...","[ребёнок, взрослый, собственный, действие, эта...","[обучение, ребёнок, действие, когнитивный, спо...","[ребёнок, способ, мочь, подход, ситуация, зави...","[ребёнок, взрослый, собственный, действие, эта...","[обучение, ребёнок, действие, когнитивный, спо...","[обучение, журнал, социальный наука, ребёнок, ..."
3,мирный атом это одна из основных надежд челове...,"атом, топливо, ядерная физика, топливные элеме...","атом, ядерная энергетика, ядерное топливо, ура...",2840,"[журнал, ядерный физика, нейтрон, толерантный ...","[поддержание цепной реакция, быстрый нейтрон, ...","[ядерный топливо, уран, урана, плутоний, плуто...","[топливо, реактор, ядерный, уран, плутоний, не...","[быстрый нейтрон, атомный энергетика, повышенн...","[ядерный топливо, уран, урана, плутоний, плуто...","[топливо, реактор, ядерный, уран, плутоний, не...","[журнал, ядерный физика, нейтрон, толерантный ..."
4,в последние годы интерес к правильному питанию...,"медицина, белки, питание, еда, калории, углево...","питание, медицина, диета, белки, жиры, углевод...",1977,"[журнал, витамин, жир, углевод, белка, калория...","[быстрый углевод, первый очередь, углевод, жир...","[углевод, человек, количество, белок, организм...","[углевод, жир, организм, человек, белок, энерг...","[быстрый углевод, первый очередь, углевод, жир...","[углевод, человек, количество, белок, организм...","[углевод, жир, организм, человек, белок, энерг...","[журнал, витамин, жир, углевод, белка, калория..."


In [108]:
def precision(best_keywords, keywords):
    precision_values = []
    for i, text in enumerate(best_keywords):
        precision_values.append(len(set(text).intersection(keywords[i]))/len(text))
    return np.array(precision_values).mean()

In [109]:
def recall(best_keywords, keywords):
    precision_values = []
    for i, text in enumerate(best_keywords):
        precision_values.append(len(set(text).intersection(keywords[i]))/len(keywords[i]))
    return np.array(precision_values).mean()

In [110]:
def fscore(best_keywords, keywords):
    precision_value = precision(best_keywords, keywords)
    recall_value = recall(best_keywords, keywords)
    return (2 * precision_value * recall_value) / (precision_value + recall_value)

## Precision

In [111]:
precision_values = []

Precision для ключевых слов без шаблонов

In [112]:
precision_values.append(precision(df['best_keywords'], df['rake_keywords']))
precision_values.append(precision(df['best_keywords'], df['textrank_keywords']))
precision_values.append(precision(df['best_keywords'], df['tfidf_keywords']))

Precision для ключевых слов с шаблононами

In [113]:
precision_values.append(precision(df['templates_best_keywords'], df['templates_rake_keywords']))
precision_values.append(precision(df['templates_best_keywords'], df['templates_textrank_keywords']))
precision_values.append(precision(df['templates_best_keywords'], df['templates_tfidf_keywords']))

## Recall

In [114]:
recall_values = []

Recall для ключевых слов без шаблонов

In [115]:
recall_values.append(recall(df['best_keywords'], df['rake_keywords']))
recall_values.append(recall(df['best_keywords'], df['textrank_keywords']))
recall_values.append(recall(df['best_keywords'], df['tfidf_keywords']))

Recall для ключевых слов с шаблононами

In [116]:
recall_values.append(recall(df['templates_best_keywords'], df['templates_rake_keywords']))
recall_values.append(recall(df['templates_best_keywords'], df['templates_textrank_keywords']))
recall_values.append(recall(df['templates_best_keywords'], df['templates_tfidf_keywords']))

## F-мера

In [117]:
fscore_values = []

F-мера для ключевых слов без шаблонов

In [118]:
fscore_values.append(fscore(df['best_keywords'], df['rake_keywords']))
fscore_values.append(fscore(df['best_keywords'], df['textrank_keywords']))
fscore_values.append(fscore(df['best_keywords'], df['tfidf_keywords']))

F-мера для ключевых слов с шаблонами

In [119]:
fscore_values.append(fscore(df['templates_best_keywords'], df['templates_rake_keywords']))
fscore_values.append(fscore(df['templates_best_keywords'], df['templates_textrank_keywords']))
fscore_values.append(fscore(df['templates_best_keywords'], df['templates_tfidf_keywords']))

## Общие результаты

In [120]:
df_scores = pd.DataFrame()
df_scores['method'] = ['rake', 'textrank', 'tfidf', 'rake_pos', 'textrank_pos', 'tfidf_pos']
df_scores['precision'] = precision_values
df_scores['recall'] = recall_values
df_scores['fscore'] = fscore_values

In [121]:
df_scores

,method,precision,recall,fscore
0,rake,0.118831,0.251273,0.161355
1,textrank,0.309740,0.033359,0.060231
2,tfidf,0.270130,0.213333,0.238395
3,rake_pos,0.120346,0.277315,0.167851
4,textrank_pos,0.317316,0.054862,0.093549
5,tfidf_pos,0.276190,0.228059,0.249827


## Описание ошибок, предложения методов

#1. Ключевые слова были выделены во всех текстах, кроме второго. Мне кажется, это связано с объемом статьи. В среднем статьи состоят из 2000 токенов. В то время, как во второй статье насчитывает 450.

#2. Самым точным методом оказался TF-IDF, думаю, это связано с тем, что слова, выбранные редакторами в качестве тегов, также были определены по частотности в тексте. Но его проблема в том, что в некоторых статьях для основного ключевого слова используется множество разных синонимов, а для второстепенных слов – нет, в связи с чем встрчается это слово очень часто. По результатам TextRank видно, что слова выделяются правильно, но находится таких слов мало (высокий относительно других precision, но низкий recall, что пагубно сказывается на общих результатах). Работает он тоже неплохо, но все-таки хуже чем TF-IDF. Rake же наоборот – находит много, но не очень точно (высокий относительно других recall, но низкий precision, но общий результат получше, чем у TextRank). Кажется, что TF-IDF – некая "золотая середина", где-то похуже других, но в целом – дает лучший результат.

#3. Кажется, чтобы улучшить работу, можно попробовать другие шаблоны (например с краткими прилагательными или еще чем-то, но точно не с глаголами). Можно сделать условие на параметр, отвечающий за встречаемость слова в тексте, в зависимости от длины данного текста. Можно поработать с другими данными, желательно из одной конкреной сферы, но данные брать из одного типа источника.